# PokeStat Guide Database Generator - Mongoose Chapter

*The Mongoose Chapter notebook is meant to focus on database preparation and generation with all data stemming from the **Pokemon.json** file.*

In [9]:
# Dependency Center
import json
import sqlite3
import pandas as pd

In [10]:
# Load the data from the JSON file
with open('static/data/pokemon.json') as f:
    data = json.load(f)

# Create a new SQLite database and establish a connection
conn = sqlite3.connect('pokedex.db')

# Create a new cursor object
c = conn.cursor()

# Create a table for Pokemon
c.execute("""
    CREATE TABLE pokemon (
        id INTEGER PRIMARY KEY,
        name TEXT,
        height INTEGER,
        weight INTEGER,
        pokemon_url TEXT
    )
""")

# Create a table for Abilities
c.execute("""
    CREATE TABLE abilities (
        pokemon_id INTEGER,
        name TEXT,
        is_hidden BOOLEAN,
        FOREIGN KEY(pokemon_id) REFERENCES pokemon(id)
    )
""")

# Create a table for Stats
c.execute("""
    CREATE TABLE stats (
        pokemon_id INTEGER,
        name TEXT,
        base_stat INTEGER,
        FOREIGN KEY(pokemon_id) REFERENCES pokemon(id)
    )
""")

# Create a table for Types
c.execute("""
    CREATE TABLE types (
        pokemon_id INTEGER,
        name TEXT,
        FOREIGN KEY(pokemon_id) REFERENCES pokemon(id)
    )
""")

# Iterate over the data and insert it into the tables
for pokemon_name, pokemon_data in data.items():
    # Insert into the Pokemon table
    c.execute("""
        INSERT INTO pokemon (id, name, height, weight, pokemon_url)
        VALUES (?, ?, ?, ?, ?)
    """, (pokemon_data['id'], pokemon_data['name'], pokemon_data['height'], pokemon_data['weight'], pokemon_data['pokemon_url']))
    
    # Get the ID of the last inserted Pokemon
    last_id = c.lastrowid
    
    # Insert abilities, stats, and types
    for ability in pokemon_data['abilities']:
        c.execute("""
            INSERT INTO abilities (pokemon_id, name, is_hidden)
            VALUES (?, ?, ?)
        """, (last_id, ability['name'], ability['is_hidden']))
        
    for stat in pokemon_data['stats']:
        c.execute("""
            INSERT INTO stats (pokemon_id, name, base_stat)
            VALUES (?, ?, ?)
        """, (last_id, stat['name'], stat['base_stat']))
        
    for type in pokemon_data['types']:
        c.execute("""
            INSERT INTO types (pokemon_id, name)
            VALUES (?, ?)
        """, (last_id, type['name']))

# Commit the changes
conn.commit()

In [13]:
# Query each table and load the data into a DataFrame
df_pokemon = pd.read_sql_query("SELECT * FROM pokemon", conn)
df_abilities = pd.read_sql_query("SELECT * FROM abilities", conn)
df_stats = pd.read_sql_query("SELECT * FROM stats", conn)
df_types = pd.read_sql_query("SELECT * FROM types", conn)

In [14]:
# Display the Pokemon, Abilities, Stats, and Types tables from the Pokemon database
# Pokemon Table: Contains the Pokemon's name, height, weight, and ID
print("Pokemon Table:")
print(df_pokemon)

Pokemon Table:
      id            name  height  weight   
0      1       bulbasaur      70      69  \
1      2         ivysaur     100     130   
2      3        venusaur     200    1000   
3      4      charmander      60      85   
4      5      charmeleon     110     190   
..   ...             ...     ...     ...   
885  886        drakloak     140     110   
886  887       dragapult     300     500   
887  888     zacian-hero     280    1100   
888  889  zamazenta-hero     290    2100   
889  890       eternatus    2000    9500   

                                           pokemon_url  
0         https://www.pokemon.com/us/pokedex/bulbasaur  
1           https://www.pokemon.com/us/pokedex/ivysaur  
2          https://www.pokemon.com/us/pokedex/venusaur  
3        https://www.pokemon.com/us/pokedex/charmander  
4        https://www.pokemon.com/us/pokedex/charmeleon  
..                                                 ...  
885        https://www.pokemon.com/us/pokedex/drakloak  


In [15]:
# Abilities Table: Contains the Pokemon's abilities and their IDs
print("\nAbilities Table:")
print(df_abilities)


Abilities Table:
      pokemon_id              name  is_hidden
0              1          overgrow          0
1              1       chlorophyll          1
2              2          overgrow          0
3              2       chlorophyll          1
4              3          overgrow          0
...          ...               ...        ...
2126         887       infiltrator          0
2127         887       cursed-body          1
2128         888    intrepid-sword          0
2129         889  dauntless-shield          0
2130         890          pressure          0

[2131 rows x 3 columns]


In [16]:
# Stats Table: Contains the Pokemon's stats and their IDs
print("\nStats Table:")
print(df_stats)


Stats Table:
      pokemon_id             name  base_stat
0              1               hp         45
1              1           attack         49
2              1          defense         49
3              1   special-attack         65
4              1  special-defense         65
...          ...              ...        ...
5335         890           attack         85
5336         890          defense         95
5337         890   special-attack        145
5338         890  special-defense         95
5339         890            speed        130

[5340 rows x 3 columns]


In [17]:
# Types Table: Contains the Pokemon's types and their IDs
print("\nTypes Table:")
print(df_types)


Types Table:
      pokemon_id      name
0              1     grass
1              1    poison
2              2     grass
3              2    poison
4              3     grass
...          ...       ...
1324         887     ghost
1325         888     fairy
1326         889  fighting
1327         890    poison
1328         890    dragon

[1329 rows x 2 columns]


In [18]:
# Close connection
conn.close()